In [ ]:
import sys
import idx2numpy
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from load_datasets import load_data_from_csv,one_hot

def acc(W,b,img,lab):
    ss = nn.Softmax(1)
    hat_label = torch.mm(img,W)+b
    sm_hat_label = ss(hat_label)
    aab = torch.sum(abs(ss(hat_label)-lab),axis=1)
    return torch.sum(abs(aab)<=1e-1)/len(img)

def acc0(W,b,img,lab):
    ss = nn.Softmax(1)
    hat_label = torch.mm(img,W)+b
    sm_hat_label=ss(hat_label)
    max_sm_hat_label=torch.max(abs(sm_hat_label),1)
    sum1=torch.sum(one_hot(max_sm_hat_label.indices)-lab<=1e-3)
    return sum1/len(img)

def acca(W,b,img,lab):
    ss = nn.Softmax(1)
    hat_label = torch.mm(img,W)+b
    sm_hat_label=ss(hat_label)
    max_sm_hat_label=torch.max(abs(sm_hat_label),1)
    sum1=torch.all(torch.equal(one_hot(max_sm_hat_label.indices),lab),dim=1)
    return sum1/len(img)
    
def test(W,b,img,lab):
    ss = nn.Softmax(1)
    img1 = img[:5000]
    lab1 = lab[:5000]
    img2 = img[5000:]
    lab2 = lab[5000:]
    acc1 = acc(W,b,img1,lab1)
    acc2 = acc(W,b,img2,lab2)
    return (acc1+acc2)/2


In [ ]:
p = 28
C = 10
N = 5000

img_tr, lab_tr, img_val, lab_val, img_test, lab_test = load_data_from_csv(0)

W = torch.rand([p**2,C],requires_grad=True)
print(W)
b = torch.rand([N,C],requires_grad=True)
la = torch.rand([N,1],requires_grad=True)
CrossEntropy = nn.CrossEntropyLoss()

optimizer = optim.Adam([W,b],lr=1e-2)
optimizer1 = optim.SGD([la], lr=1e-2, momentum=0.9)

lr = 1e-2
for k in range(500):
    optimizer1.zero_grad()
    for i in range(20):
        optimizer.zero_grad()
        hat_label = torch.mm(img_tr,W)+b
        loss1 = CrossEntropy(la*hat_label, lab_tr)
        #if i % 20 == 0:
            #print(f'loss1={loss1}')
        loss1.backward()
        optimizer.step()

    hat_label2 = torch.mm(img_val,W)+b
    loss2 = CrossEntropy(hat_label2, lab_val)+ 1e-3*torch.norm(W,'fro')**2
    loss2.backward()
    if k % 50 == 0:
        print([f'acc={test(W,b,img_test,lab_test)}',f'loss2={loss2}'])
    with torch.no_grad():
        la -= lr * la.grad
    #optimizer1.step() 


In [ ]:
import torch

# 假设 outputs 是你模型的输出, targets 是真实的标签
outputs = torch.tensor([[0.1, 0.2, 0.7], [0.7, 0.2, 0.1]])
targets = torch.tensor([2, 0])

# 获得最大概率对应的类别
_, predicted = torch.max(outputs, 1)
print(predicted)
# 比较预测结果与真实标签
correct = (predicted == targets).sum().item()

print(f'正确标签数：{correct}')
print(f'总标签数：{targets.size(0)}')
print(f'准确率：{correct / targets.size(0)}')

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

img_tr, lab_tr, img_val, lab_val, img_test, lab_test = load_data_from_csv()

# 看图的部分
plt.imshow(img_tr[1].reshape([-1,28]))
plt.title(lab_tr[1])

In [ ]:
len(img_val)

In [ ]:
ss = nn.Softmax(1)
hat_label = torch.mm(img_val,W)+b
sm_hat_label=ss(hat_label)
print(f'sm_hat_label size{sm_hat_label.size()}')
max_sm_hat_label=torch.max(abs(sm_hat_label),1)
print(f'max_sm_hat_label.indices{max_sm_hat_label.indices.size()}')
sum1=torch.sum(abs(one_hot(max_sm_hat_label.indices)-lab_val)<=1e-3)
print(f'one_hot(max_sm_hat_label.indices)={one_hot(max_sm_hat_label.indices)}')

In [ ]:
print(torch.sum(torch.sum(abs(one_hot(max_sm_hat_label.indices)-lab_val),1)-2==0))

In [ ]:
a=torch.tensor([1==1, 2==2])
torch.sum(a)

In [ ]:
W = torch.rand([p**2,C],requires_grad=True)
b = torch.rand([N,C],requires_grad=True)
la = torch.rand([N,1],requires_grad=True)
CrossEntropy = nn.CrossEntropyLoss()
optimizer = optim.Adam([W,b],lr=1e-2)

for i in range(500):
    optimizer.zero_grad()
    hat_label = torch.mm(img_tr,W)+b
    loss1 = CrossEntropy(la*hat_label, lab_tr)
    if i % 50 == 0:
        print(f'hat_label={ss(hat_label)},lab_tr={lab_tr}')
        print(f'loss1={loss1},acc={acc(W,b,img_val,lab_val)}')
    loss1.backward()
    optimizer.step()